In [1]:
!pip install geemap --upgrade
!pip install earthengine-api --upgrade

In [2]:
import ee

In [3]:
import geemap

In [4]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [5]:
map_mrmra = geemap.Map()
point = ee.Geometry.Point([27.8624, 40.4546])

image = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
    .filterBounds(point)  \
    .filterDate("2018-01-01", "2019-12-31") \
    .sort("CLOUD_COVER") \
    .first()  \
    .select("B[1-7]")

vis_parameters = {"min": 0, "max": 3000, "bands": ["B4", "B3", "B2"]}

map_mrmra.centerObject(point, 8)
map_mrmra.addLayer(image, vis_parameters, "Landsat-8")
map_mrmra

Map(center=[40.4546, 27.8624], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [6]:
ee.Algorithms.If(ee.List(image.propertyNames()).contains("system:time_start"), True, False).getInfo()

True

In [7]:
props = geemap.image_props(image)
props.getInfo()

{'CLOUD_COVER': 0.02,
 'CLOUD_COVER_LAND': 0.01,
 'EARTH_SUN_DISTANCE': 1.016671,
 'ESPA_VERSION': '2_23_0_1a',
 'GEOMETRIC_RMSE_MODEL': 6.812,
 'GEOMETRIC_RMSE_MODEL_X': 4.902,
 'GEOMETRIC_RMSE_MODEL_Y': 4.731,
 'IMAGE_DATE': '2018-07-03',
 'IMAGE_QUALITY_OLI': 9,
 'IMAGE_QUALITY_TIRS': 9,
 'LANDSAT_ID': 'LC08_L1TP_181032_20180703_20180717_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1531790086000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
 'SATELLITE': 'LANDSAT_8',
 'SENSING_TIME': '2018-07-03T08:51:03.0197999Z',
 'SOLAR_AZIMUTH_ANGLE': 127.869438,
 'SOLAR_ZENITH_ANGLE': 24.914536,
 'SR_APP_VERSION': 'LaSRC_1.3.0',
 'WRS_PATH': 181,
 'WRS_ROW': 32,
 'system:asset_size': '581.571197 MB',
 'system:band_names': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_181032_20180703',
 'system:index': 'LC08_181032_20180703',
 'system:time_end': '2018-07-03 08:51:03',
 'system:time_start': '2018-07-03 08:51:03',
 'system:version': 153242007

In [8]:
props.get("IMAGE_DATE").getInfo()

'2018-07-03'

In [9]:
props.get("CLOUD_COVER").getInfo()

0.02

In [10]:
#Map_1.user_roi.getInfo()

In [11]:
#region = Map_1.user_roi

In [12]:
training = image.sample(**{
#    "region": region,
    "scale": 30,
    "numPixels": 5000,
    "seed": 0,
    "geometries": True
})
map_mrmra.addLayer(training, {}, "training", False)
map_mrmra

Map(center=[40.4546, 27.8624], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [13]:
n_clusters = 4
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [14]:
result= image.cluster(clusterer)

map_mrmra.addLayer(result.randomVisualizer(), {}, "clusters")
map_mrmra

Map(center=[40.4546, 27.8624], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [15]:
legend_keys = ["Water", "Agricultural", "Forest", "Agriculture"]
legend_colors = ["#3CC7FF", "#FF4848", "#009612", "#FFFFB3"]
result = result.remap([0, 1, 2, 3], [1, 2, 3, 4])

map_mrmra.addLayer(result, {"min": 1, "max": 4, "palette": legend_colors}, "Labelled Clusters")
map_mrmra.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, poisiton="bottomright")
map_mrmra

Map(center=[40.4546, 27.8624], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [16]:
print("Change Layer Opacity:")
cluster_layer = map_mrmra.layers[-1]
cluster_layer.interact(opacity = (0, 1, 0.1))

Change Layer Opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [17]:
import os
out_dir = os.path.join(os.path.expanduser("Output"))
out_file = os.path.join(out_dir, "cluster.tif")

In [18]:
geemap.ee_export_image(result, filename=out_file, scale=90)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\bltse\OneDrive\Masaüstü\STRS_hw\Output\cluster.tif
